<a href="https://colab.research.google.com/github/ParkSeonungHun/Hun/blob/main/python_basic/%EB%B3%B5%EC%9E%A1%ED%95%98%EA%B3%A0_%EC%9D%98%EB%AF%B8%EC%9E%88%EB%8A%94_%EA%B7%B8%EB%A6%BC_%EA%B5%AC%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 복잡하고 의미있는 그림 구성


**경고**

이 튜토리얼은 실험적 / 임시적 API를 문서화합니다. 사용자 피드백을 받기 위해 v3.3에서이 기능을 출시하고 있습니다. 경고없이 향후 버전에서 중대한 변경을 할 수 있습니다.

그림의 축을 균일하지 않은 그리드로 배치하는 것은 지루하고 장황 할 수 있습니다. 밀도가 높고 균일 한 그리드 Figure.subplots의 경우 레이아웃의 여러 열 / 행에 걸쳐 있거나 그림의 일부 영역을 비워 두는 축과 같이 더 복잡한 레이아웃의 경우 gridspec.GridSpec( GridSpec 및 기타 기능을 사용하여 그림 레이아웃 사용자 지정 참조 ) 또는 수동으로 사용할 수 있습니다. 축을 배치하십시오. Figure.subplot_mosaic은 (는)이 프로세스를 간소화하기 위해 축 (ASCII 아트 또는 중첩 목록)을 시각적으로 배치하는 인터페이스를 제공하는 것을 목표로합니다.

이 인터페이스는 자연스럽게 축 이름 지정을 지원합니다. Figure.subplot_mosaicFigure를 배치하는 데 사용되는 레이블에 키가있는 사전을 반환합니다. 이름이있는 데이터 구조를 반환함으로써 Figure 레이아웃과 독립적 인 플로팅 코드를 작성하는 것이 더 쉽습니다.

이것은 제안 된 MEP 및 R 용 패치 워크 라이브러리에서 영감을 얻었습니다 . 연산자 오버로딩 스타일을 구현하지 않지만 (중첩 된) Axes 레이아웃을 지정하기위한 Pythonic API를 제공합니다.

In [1]:
import matplotlib.pyplot as plt
import numpy as np


# Helper function used for visualization in the following examples
def identify_axes(ax_dict, fontsize=48):
    """
    Helper to identify the Axes in the examples below.

    Draws the label in a large font in the center of the Axes.

    Parameters
    ----------
    ax_dict : Dict[str, Axes]
        Mapping between the title / label and the Axes.

    fontsize : int, optional
        How big the label should be
    """
    kw = dict(ha="center", va="center", fontsize=fontsize, color="darkgrey")
    for k, ax in ax_dict.items():
        ax.text(0.5, 0.5, k, transform=ax.transAxes, **kw)

2x2 그리드를 원하면 플로팅을 위해 인덱싱 할 수 Figure.subplots있는 2D 배열을 반환하는 데 사용할 수 있습니다 axes.Axes.

In [ ]:
np.random.seed(19680801)
hist_data = np.random.randn(1_500)


fig = plt.figure(constrained_layout=True)
ax_array = fig.subplots(2, 2, squeeze=False)

ax_array[0, 0].bar(['a', 'b', 'c'], [5, 7, 9])
ax_array[0, 1].plot([1, 2, 3])
ax_array[1, 0].hist(hist_data, bins='auto')
ax_array[1, 1].imshow([[1, 2], [2, 1]])

identify_axes(
    {(j, k): a for j, r in enumerate(ax_array) for k, a in enumerate(r)}
)    

사용 Figure.subplot_mosaic하면 동일한 레이아웃을 생성 할 수 있지만 축에 의미 론적 이름을 지정할 수 있습니다.

In [ ]:
fig = plt.figure(constrained_layout=True)
ax_dict = fig.subplot_mosaic(
    [['bar',  'plot'],
     ['hist', 'image']])
ax_dict['bar'].bar(['a', 'b', 'c'], [5, 7, 9])
ax_dict['plot'].plot([1, 2, 3])
ax_dict['hist'].hist(hist_data)
ax_dict['image'].imshow([[1, 2], [2, 1]])
identify_axes(ax_dict)

간의 주요 차이 Figure.subplots및 Figure.subplot_mosaic리턴 값이다. 전자는 인덱스 액세스를 위해 배열을 반환하고 후자는 axes.Axes생성 된 인스턴스에 레이블을 매핑하는 사전을 반환합니다.

In [ ]:
print(ax_dict)

## 문자열

축 레이블을 단일 문자로 제한하여 사용할 수 있습니다. 사용하여 "ASCII 아트"로 원하는 축을 "그릴"수 있습니다. 다음과 같은


In [13]:
layout = """
    AB
    CD
    """

2x2 그리드에 배치 된 4 개의 축을 제공하고 위와 동일한 그림 레이아웃을 생성합니다 (이제 대신으로 레이블이 지정됨 ).{"A", "B", "C", "D"}{"bar", "plot", "hist", "image"}

In [ ]:
fig = plt.figure(constrained_layout=True)
ax_dict = fig.subplot_mosaic(layout)
identify_axes(ax_dict)

우리가 할 수있는 일 Figure.subplot_mosaic당신이 할 수없는 것을는 Figure.subplots축이 여러 행 또는 열을 걸쳐하도록 지정합니다.

4 개의 축을 다시 배열하여 C가 하단의 수평 범위이고 D가 오른쪽의 수직 범위가되도록하려면 다음을 수행합니다.

In [ ]:
axd = plt.figure(constrained_layout=True).subplot_mosaic(
    """
    ABD
    CCD
    """
)
identify_axes(axd)

그림의 모든 공간을 축으로 채우지 않으려면 그리드의 일부 공간을 공백으로 지정할 수 있습니다.

In [ ]:
axd = plt.figure(constrained_layout=True).subplot_mosaic(
    """
    A.C
    BBB
    .D.
    """
)
identify_axes(axd)

마침표가 아닌 다른 문자를 사용 "."하여 빈 공간을 표시하려는 경우 empty_sentinel 을 사용하여 사용할 문자를 지정할 수 있습니다.

In [ ]:
axd = plt.figure(constrained_layout=True).subplot_mosaic(
    """
    aX
    Xb
    """,
    empty_sentinel="X",
)
identify_axes(axd)

내부적으로 우리가 사용하는 문자에는 의미가 없으며 모든 유니 코드 코드 포인트가 유효합니다!

In [ ]:
axd = plt.figure(constrained_layout=True).subplot_mosaic(
    """αб
       ℝ☢"""
)
identify_axes(axd)

공백은 입력을 처리하는 동안 제거 될 수 있으므로 공백을 레이블로 사용하거나 문자열 속기 빈 센티넬로 사용하지 않는 것이 좋습니다.

## 레이아웃 및 서브 플롯 생성 제어

이 기능은 위에 구축되었으며 gridspec키워드 인수를 기본 gridspec.GridSpec (과 동일 Figure.subplots)에 전달할 수 있습니다 .

이 경우 입력을 사용하여 배열을 지정하고 싶지만 gridspec_kw 를 통해 행 / 열의 상대적 너비를 설정합니다 .

In [ ]:
axd = plt.figure(constrained_layout=True).subplot_mosaic(
    """
    .a.
    bAc
    .d.
    """,
    gridspec_kw={
        # set the height ratios between the rows
        "height_ratios": [1, 3.5, 1],
        # set the width ratios between the columns
        "width_ratios": [1, 3.5, 1],
    },
)
identify_axes(axd)

또는 { left , right , bottom , top } 키워드 인수를 사용하여 전체 레이아웃의 위치를 ​​지정하여 동일한 레이아웃의 여러 버전을 Figure에 넣습니다.

In [12]:
layout = """AA
            BC"""
fig = plt.figure()
axd = fig.subplot_mosaic(
    layout,
    gridspec_kw={
        "bottom": 0.25,
        "top": 0.95,
        "left": 0.1,
        "right": 0.5,
        "wspace": 0.5,
        "hspace": 0.5,
    },
)
identify_axes(axd)

axd = fig.subplot_mosaic(
    layout,
    gridspec_kw={
        "bottom": 0.05,
        "top": 0.75,
        "left": 0.6,
        "right": 0.95,
        "wspace": 0.5,
        "hspace": 0.5,
    },
)
identify_axes(axd)


AttributeError: ignored

<Figure size 432x288 with 0 Axes>

서브 플롯을 만드는 데 사용되는 인수를 전달할 수도 있습니다 (다시 말하지만 Figure.subplots).

In [ ]:
axd = plt.figure(constrained_layout=True).subplot_mosaic(
    "AB", subplot_kw={"projection": "polar"}
)
identify_axes(axd)

## 중첩 목록 입력

예를 들어 spans, blanks 및 gridspec_kw 사용하여 목록을 전달할 때 (내부적으로 문자열 속기를 중첩 된 목록으로 변환) 문자열 속기로 할 수있는 모든 작업을 수행 할 수 있습니다 .

In [ ]:
axd = plt.figure(constrained_layout=True).subplot_mosaic(
    [["main", "zoom"],
     ["main", "BLANK"]
     ],
    empty_sentinel="BLANK",
    gridspec_kw={"width_ratios": [2, 1]}
)
identify_axes(axd)

또한 목록 입력을 사용하여 중첩 된 레이아웃을 지정할 수 있습니다. 내부 목록의 모든 요소는 다른 중첩 목록 집합이 될 수 있습니다.

In [ ]:
inner = [
    ["inner A"],
    ["inner B"],
]

outer_nested_layout = [
    ["main", inner],
    ["bottom", "bottom"],
]
axd = plt.figure(constrained_layout=True).subplot_mosaic(
    outer_nested_layout, empty_sentinel=None
)
identify_axes(axd, fontsize=36)

2D NumPy 배열을 전달하여 다음과 같은 작업을 수행 할 수도 있습니다.

In [ ]:
layout = np.zeros((4, 4), dtype=int)
for j in range(4):
    layout[j, j] = j + 1
axd = plt.figure(constrained_layout=True).subplot_mosaic(
    layout, empty_sentinel=0
)
identify_axes(axd)